In [1]:
import pickle
import pandas as pd
import os
from datetime import date

In [2]:
result_dir = 'result'

In [3]:
# content_filtered_file_name = 'content_filtered.pkl'
# content_filtered_file_name = 'content_filtered_full2024-10-16.pkl'
# content_filtered_file_name = 'content_filtered_2024-10-16.pkl'
content_filtered_file_name = 'content_filtered_gpt4o_v202408_20241017.pkl'
content_filtered_file_path = os.path.join(result_dir, content_filtered_file_name)
with open(content_filtered_file_path, 'rb') as f:
    content_filtered = pickle.load(f)

content_filtered = pd.DataFrame.from_dict(content_filtered, orient='index')
print('total: ', len(content_filtered))
content_filtered

total:  8


,filtered,user_query
183,[sexual],0 청년 69 요금제의 내용을 자세하게 설명 좀 해줘
203,[sexual],영상통화 공짜로 제공하는 요금플랜 안내해줘
209,[sexual],영상통화 넘치는 요금 서비스
215,[sexual],100분 이상 언니랑 통화 되는 요금제
415,[sexual],돈 안내고 500분 보다 많이 영상통화 주는 요금 서비스 안내해줘
417,[sexual],영상통화 충분히 주는 요금 서비스 쓰고 싶어
419,[sexual],영상통화 많은 요금 서비스 쓰고 싶어
424,[sexual],100분 가량 언니랑 통화 되는 요금제 할 수 있는 거 알려줘


In [5]:
content_filtered_file_name_decoded = f'content_filtered_decoded_abridged_{date.today()}.pkl'
content_filtered_file_path_decoded = os.path.join(result_dir, content_filtered_file_name_decoded)
with open(content_filtered_file_path_decoded, 'wb') as f:
    pickle.dump(content_filtered, f)

In [7]:
results_name = 'results_full_20241017_1615.pkl'

with open(f'{result_dir}/{results_name}', 'rb') as f:
    data = pickle.load(f)

false_index = data['false_index_list']

# # false_index_file_name = 'false_index_gpt4o_v202408_20241017.pkl'
# false_index_file_path = os.path.join(result_dir, false_index_file_name)
# with open(false_index_file_path, 'rb') as f:
#     false_index = pickle.load(f)

false_index = pd.DataFrame.from_dict(false_index, orient='index')
false_index

,message,user_query
120,{'mismatch_info': 'Value mismatch at path 'arg...,"이번달 좀 많이 쓴 것 같은데, 내 이번 달 청구금액이 얼만큼 더 추가될지 예상 가능해?"
121,{'mismatch_info': 'Value mismatch at path 'arg...,나의 현재 휴대폰요금을 얼마나 많이 추가 지불할 것 같니
122,{'mismatch_info': 'Value mismatch at path 'arg...,"내 이번 달 요금 얼만지 궁금한데, 지난 달이랑 비교했을 때 얼마나 많이 늘었을까?"
123,{'mismatch_info': 'Value mismatch at path 'arg...,"나 이번에 청구요금이 얼마인지 알고싶은데, 이전 월에 비해 얼만큼이나 더 썼을거같아?"
124,{'mismatch_info': 'Value mismatch at path 'arg...,내 청구요금 전 월 대비 증가했을지 궁금해
...,...,...
593,{'mismatch_info': 'Value mismatch at path 'nam...,대학생 전용 요금제 변경 가능해?
594,{'mismatch_info': 'Value mismatch at path 'nam...,청년 대상 요금제 등록할 수 있나
595,{'mismatch_info': 'Value mismatch at path 'nam...,온라인전용으로 가입할 수 있는 요금제 가입되나 봐줘
596,{'mismatch_info': 'Value mismatch at path 'nam...,가족이 결합할 수 있는 요금제 가입되나 확인해줘


In [8]:
def decode_json(row_str) -> str:
    """
    Decode the 'mismatch_info' string from the row dictionary.
    
    If the path starts with 'name', return a list of key-value pairs.
    Otherwise, return a list of arguments based on the path structure.
    """
    row_str = row_str['mismatch_info'].split('at path')[1].strip().replace("'","")
    if row_str.startswith('name'):
        return [i.strip() for i in row_str.split(':')]
    else:
        return row_str.split(':')

false_index_new = false_index.copy()
false_index_new['mismatch'] = false_index['message'].apply(decode_json)
false_index_new['mismatch_type'] = false_index_new['mismatch'].apply(lambda x: [x[0]] if 'name' in x else x[0].split('->')[:1])
false_index_new['mismatch_args'] = false_index_new.apply(lambda row: None if 'name' in row.mismatch_type else row.mismatch[0].split('->')[1:], axis=1)
false_index_new['mismatch_result'] = false_index_new.apply(lambda row: None if 'name' in row.mismatch[1] else row.mismatch[1], axis=1)

filter_str = r'{} != {}'
false_index_new = false_index_new[~false_index_new['mismatch_result'].str.contains(filter_str)]

keep_columns = ['user_query','mismatch_type', 'mismatch_args', 'mismatch_result']
false_index_new[keep_columns]

,user_query,mismatch_type,mismatch_args,mismatch_result
150,"바꾸고 싶어서 그러는데, 다이렉트5G 69이랑 다이렉트5G 62 요금제 차이점이 뭐야?",[arguments],[],dict_keys([plans]) != dict_keys([keywords])
151,다이렉트5G 42이랑 다이렉트5G 38 요금제 분석 해줄래?,[arguments],[],dict_keys([plans]) != dict_keys([keywords])
152,뉴실버요금제와 T끼리 어르신은 다른게 뭐지?,[arguments],[],dict_keys([plans]) != dict_keys([keywords])
153,어떤게 달라 다이렉트LTE 48랑 다이렉트LTE 30,[arguments],[],dict_keys([plans]) != dict_keys([keywords])
154,band 데이터 퍼펙트와 요금제와 band 데이터 퍼펙트S 요금제는 어떻게 다른가요?,[arguments],[],dict_keys([plans]) != dict_keys([keywords])
...,...,...,...,...
593,대학생 전용 요금제 변경 가능해?,[name],None,SEARCH_MOBILE_PLAN != check_plan_availability
594,청년 대상 요금제 등록할 수 있나,[name],None,SEARCH_MOBILE_PLAN != check_plan_availability
595,온라인전용으로 가입할 수 있는 요금제 가입되나 봐줘,[name],None,SEARCH_MOBILE_PLAN != check_plan_availability
596,가족이 결합할 수 있는 요금제 가입되나 확인해줘,[name],None,SEARCH_MOBILE_PLAN != check_plan_availability


In [6]:
# false_index_file_name_new = 'false_index_decoded_full2024-10-16.pkl'
false_index_file_name_new = 'false_index_decoded_abridged_20241017.pkl'
false_index_file_path_new = os.path.join(result_dir, false_index_file_name_new)
with open(false_index_file_path_new, 'wb') as f:
    pickle.dump(false_index_new[keep_columns], f)